In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

/home/kf7mxe/anaconda3/envs/time-series-data-mining/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# import data.tsv file and create a dataframe

df = pd.read_csv('data.tsv', sep='\t')

# split df into training and test sets

train_df = df.sample(frac=0.8, random_state=0)
test_df = df.drop(train_df.index)

train_x = train_df.drop(0, axis=1)
train_y = train_df[0]

test_x = test_df.drop(0, axis=1)
test_y = test_df[0]

# train a model pytorch recurrent neural network



# input_size = 

class SitUpDetector(nn.Module):
    def __init__(self,input_size, output_size, hidden_size, num_layers):
        super(SitUpDetector, self).__init__()
        self.hidden_dim = hidden_size
        self.num_layers = num_layers

        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        batch_size = x.size(0)

        hidden = self.init_hidden(batch_size)

        out, hidden = self.rnn(x, hidden)

        out = self.fc1(out)

        return out, hidden
    
    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_dim)
        return hidden

model = SitUpDetector()
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

for epoch in range(1000):
    optimizer.zero_grad()
    output = model(train_x)
    loss = criterion(output, train_y)
    loss.backward()
    optimizer.step()
    if epoch % 100 == 0:
        print(loss.item())

# test the model

with torch.no_grad():
    output = model(test_x)
    print(output)
    print(test_y)


# save the model

torch.save(model.state_dict(), 'sit_up_detector.pth')




